In [1]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict, namedtuple
from tqdm import tqdm
from unidecode import unidecode
import os
import pickle as pk
import json
import random
import string
from tqdm import tqdm

In [2]:
with open('metadata.txt', 'r') as f:
    start_page = int(f.read()) + 1
start_page

1330

In [3]:
def id_generator(size=10, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [4]:
def extract(url):
    content = requests.get(url).content
    soup = BeautifulSoup(content, "html.parser")
    try:
        title = soup.find_all('h1', {'class': 'title'})[0].text.strip()
    except:
        title = ''
    try:
        text = soup.find_all('div', {'class': 'item-text'})[0].text
    except:
        text = ''
    try:
        lis = soup.find_all('section', {'class': 'box tags'})[0].find_all('li')
        tags = [li.text for li in lis]
    except:
        tags = []
    try:
        comments = []
        comments_ = soup.find_all('div', {'class': 'comments-list'})[0].find_all('li')
        for comment_ in comments_:
            plus = comment_.find_all('div', {'class':'rating'})[0].find_all('a')[0].text
            minus = comment_.find_all('div', {'class':'rating'})[0].find_all('a')[1].text
            comment_rating = [int(minus), int(plus)]
            comment_text = comment_.find_all('div', 'comment-body')[0].text
            comments.append({
                'text': comment_text,
                'rating': comment_rating
            })
        num_comment = len(comments)
    except:
        num_comment = 0
        comments = []
    try:
        category = soup.find_all('ol', {'class': 'breadcrumb'})[0].find_all('li')[1].text
    except:
        category = ''
    
    return text, title, tags, category, num_comment, comments

In [5]:
def write_file(dirname, link, title, text, tags, category, num_comment, comments):
    data = {
        'link': link,
        'title': title,
        'text': text,
        'tags': tags,
        'category': category,
        'num_comment': num_comment,
        'comments': comments
    }
    filename = os.path.join(dirname, id_generator())
    with open(os.path.join(filename), 'w', encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False)

In [6]:
random.seed(12345)
A = [i for i in range(2000)]
random.seed(10000)
random.shuffle(A)

In [7]:
links = []
for idx, i in tqdm(enumerate(A[start_page: ])):
    news_links = list()
    khabar_online_main = requests.get(f'https://www.khabaronline.ir/page/search.xhtml?q=%D8%AE%D8%A8%D8%B1&a=0&alltp=true&allpl=true&pi={i}&allsr=true&pageSize=20&allty=true').content
    soup = BeautifulSoup(khabar_online_main, "html.parser")
    divs = soup.find_all('div', {'class': 'desc'})
    for div in divs:
        link = div.find_all('h3')[0].find_all('a')[0]
        news_links.append(link)
    for news_link in news_links:
        link = 'https://www.khabaronline.ir' + news_link['href']
        links.append(link)
        text, title, tags, category, num_comment, comments = extract(link)
        if num_comment > 2:
            print(num_comment, link)
            write_file('pos_news', link, title, text, tags, category, num_comment, comments)
        else:
            write_file('all_news', link, title, text, tags, category, num_comment, comments)
    with open('metadata.txt', 'w', encoding="utf-8") as f:
        f.write(str(start_page + idx))

0it [00:00, ?it/s]

22 https://www.khabaronline.ir/news/1541330/خبر-مهم-نوبخت-درباره-همسان-سازی-حقوق-بازنشستگان
17 https://www.khabaronline.ir/news/1541348/واکنش-تند-اسرائیل-به-حضور-انریکه-مورا-در-مراسم-تحلیف-رئیسی
21 https://www.khabaronline.ir/news/1541338/ارشا-اقدسی-را-شبانه-کشتند
24 https://www.khabaronline.ir/news/1541303/ببینید-صحبت-های-تلخ-ارشا-اقدسی-درباره-مرگ-دوست-ندارم-روی


1it [00:08,  8.67s/it]

21 https://www.khabaronline.ir/news/1643424/ببینید-وعده-جالب-وزیر-به-مردم-خیال-مستاجرین-تخت-نگران-قرارداد
4 https://www.khabaronline.ir/news/1643411/صداوسیما-گرفتار-روایت-های-یک-سویه-این-رونالدو-هندی-است
10 https://www.khabaronline.ir/news/1643432/موجر-و-مستاجر-با-هم-دعوا-ندارند-هردوشان-با-دولت-دعوا-دارند
33 https://www.khabaronline.ir/news/1643420/ببینید-درخواست-جالب-سردار-آزمون-از-پدر-زن-آینده-آناهیتا-خانم


2it [00:22, 11.79s/it]

12 https://www.khabaronline.ir/news/1643436/مابه-التفاوت-افزایش-حقوق-فروردین-و-اردیبهشت-بازنشستگان-کی-واریز
6 https://www.khabaronline.ir/news/1724894/جدیدترین-قیمت-گوشت-گوسفندی-در-بازار-ران-گوسفندی-کیلویی-چند
3 https://www.khabaronline.ir/news/1724914/مرندی-هفته-خوبی-برای-رژیم-صهیونیستی-نیست
37 https://www.khabaronline.ir/news/1724708/شما-نظر-بدهید-آیا-در-معابر-و-خیابان-های-تان-با-خاموشی-روبه-رو
3 https://www.khabaronline.ir/news/1724778/جانشین-کی-روش-مرد-همیشه-نایب-قهرمان


3it [00:33, 11.16s/it]

9 https://www.khabaronline.ir/news/1603592/سیگنال-منابع-غربی-به-بازار-ارز-دلار-چقدر-ریخت
18 https://www.khabaronline.ir/news/1603638/وقتی-بابا-رفت-پرسپولیس-بی-خانمان-شد
10 https://www.khabaronline.ir/news/343557/خبری-تازه-درباره-ساخت-فصل-چهارم-سریال-نون-خ
35 https://www.khabaronline.ir/news/1603553/خبر-جالب-برای-بازنشستگان-امشب-جیب-بازنشسته-ها-پر-پول-می-شود


4it [00:44, 11.12s/it]